In [ ]:
%cd /gxfs_work/cau/sunms534/trading_bot/

In [ ]:
import pandas as pd
from src.config import config 
from tqdm.notebook import tqdm
tqdm.pandas()

pd.set_option(
    'display.max_colwidth', 2000
)

## Experimental

In [ ]:
news_csv = pd.read_csv(config.data.fnspid.raw)

In [ ]:
news_csv.rename(columns={
    "Date": "time",
    "Stock_symbol": "stocks",
    "Author": "author",
    "Article_title": "title",
    "Article":"body"
}, inplace=True)

In [ ]:
N_0 = news_csv.shape[0]

In [ ]:
news_csv = news_csv.loc[:, ['time', 'stocks', 'author', 'title', 'body']]

In [ ]:
# Inspect rows with insufficient information 
news_csv[news_csv[['time', 'stocks', 'body']].isna().any(axis=1) & ~news_csv[['time', 'stocks', 'body']].isna().all(axis=1)].head()

In [ ]:
news_csv.dropna(subset=['time', 'stocks', 'body'], inplace=True)

In [ ]:
N_1 = news_csv.shape[0]

In [ ]:
print(f"{N_0=}, {N_1=}")
print(f"Dropped {N_0 - N_1} entries due to NaNs.")

In [ ]:
news_csv['time'] = pd.to_datetime(news_csv['time'])

Many time stamps don't contain intra-day information. Hence we need to set the intra-day time of these entries to the end of the day and mark them as special rows.

In [ ]:
news_csv['time'].dt.hour.value_counts().sort_index().head(5)

In [ ]:
x = news_csv['time'].dt
no_intra_day_entries_mask = (x.hour == 0) & (x.minute == 0)

In [ ]:
# Inspect entries which we will modify
# news_csv.loc[no_intra_day_entries_mask, :].head()

In [ ]:
news_csv["intra_day_time"] = True
news_csv.loc[no_intra_day_entries_mask, "intra_day_time"] = False

In [ ]:
# Transform date into time by offsetting by one day
news_csv.loc[no_intra_day_entries_mask, "time"] = news_csv.loc[no_intra_day_entries_mask, "time"] + pd.offsets.Day(1)

In [ ]:
news_csv.to_parquet(config.data.fnspid.raw_html_parsed)